# Comparison between patients and controls

In this notebook we made the analysis for matched data according to `sex` and `age_at_onset` variables. The summary statistics compare patients and controls. 

In [1]:
# imports

import os
import sys
sys.path.append('/mnt/0A2AAC152AABFBB7/CGE/luxgiant-clinical')
import luxgiant_clinical.MaleFemale as mf

import pandas as pd

In [2]:
# load data

folder_path = '/mnt/0A2AAC152AABFBB7/CGE/luxgiant-clinical/data'

df = pd.read_csv(
    os.path.join(folder_path, 'cleaned_file.csv'), 
    index_col=0, low_memory=False
)
matched_ids = pd.read_csv(
    os.path.join(folder_path, 'matched_ids.csv'), 
    index_col=0, low_memory=False
)
df = df.merge(matched_ids, on='participant_id')

In [3]:
# fill age at onset for controls as age at assessment

mask_control = (df['Status']=='Control')

df.loc[mask_control, "age_at_onset"] = df.loc[mask_control,"age"]

## Summary statistics for demography and lifestyle

In [4]:
# rename and select first group of variables

variables_dict_1 = {
    "age"                       : "Age at Recruitment (years)",
    "age_at_onset"              : "Age at Onset (years)" ,
    "agecat_1"                  : "Onset <50 years" ,
    "agecat_2"                  : "Onset <40 years",
    "PD_duration"               : "Duration of Disease (years)",
    "years_of_education"        : "Years of Education",
    "bmi_comp"                  : "Body Mass Index",
    "work"                      : "Currently Employed",
    "nature_of_work___1"        : "Agriculture as a job",
    "over_your_lifetime_have_yo": "Life time direct exposure to pesticide/insecticide/fungicide",
    "during_your_lifetime_did_y": "Exposure to chemicals to kill insects in your life time",
    "in_your_lifetime_have_you" : "Smoked 100 or more cigarettes in lifetime",
    "in_your_lifetime_have"     : "Regular consumption of caffeinated drinks for >6months",
    "have_you_ever_had_a_head_i": "Head injury or concussion",

}
variables_1 = ['participant_id', 'Status'] + list(variables_dict_1.keys())

# statistical measures
stats_meas = {
    'mean'  : ["Age at Recruitment (years)", "Age at Onset (years)", "Body Mass Index"],
    'n'     : ["Onset <50 years", "Onset <40 years", "Currently Employed", "Agriculture as a job", 
               "Life time direct exposure to pesticide/insecticide/fungicide", "Exposure to chemicals to kill insects in your life time",
               "Smoked 100 or more cigarettes in lifetime", "Regular consumption of caffeinated drinks for >6months",
               "Head injury or concussion"],
    'median': ["Duration of Disease (years)", "Years of Education"]
}

In [ ]:
# keep only variables for analysis

df_1 = df[variables_1].copy()
df_1 = df_1.rename(columns=variables_dict_1)

In [5]:
# recode variable values

df_1["Onset <50 years"]     = df_1["Onset <50 years"].map({'<50':1, '>=50':0})
df_1["Onset <40 years"]     = df_1["Onset <40 years"].map({'<40':1, '>=40':0})
df_1["Currently Employed"]  = df_1["Currently Employed"].map({'Yes':1, 'No':0})
df_1["Agriculture as a job"]= df_1["Agriculture as a job"].map({'Checked':1, 'Unchecked':0})

df_1["Life time direct exposure to pesticide/insecticide/fungicide"] = \
    df_1["Life time direct exposure to pesticide/insecticide/fungicide"]\
        .map({'Yes':1, 'No':0, 'Dont Know':0, 'Refused':0})
df_1["Exposure to chemicals to kill insects in your life time"] =\
    df_1["Exposure to chemicals to kill insects in your life time"]\
        .map({'Yes':1, 'No':0, 'Dont Know':0, 'Refused':0})
df_1["Smoked 100 or more cigarettes in lifetime"] = \
    df_1["Smoked 100 or more cigarettes in lifetime"]\
        .map({'Yes':1, 'No':0, 'Dont Know':0, 'Refused':0})
df_1["Regular consumption of caffeinated drinks for >6months"] = \
    df_1["Regular consumption of caffeinated drinks for >6months"]\
        .map({'Yes':1, 'No':0, 'Dont Know':0, 'Refused':0})
df_1["Head injury or concussion"] = \
    df_1["Head injury or concussion"].map({'Yes':1, 'No':0, 'Dont Know':0, 'Refused':0})

In [6]:
# create empty dataframe for summary statistics

summary_cols = ['Variable', 'Statistical Measure', 'Patient', 'Control', 'Available Sample for Analysis']
df_summary = pd.DataFrame(columns=summary_cols)

In [7]:
# mean and standard deviation of some continuous variables

res1 = mf.mean_std(df_1, stats_meas['mean'], grouping_by='Status')

df_summary1 = mf.summaryze_mean_std(df_summary, res1, stats_meas['mean'], 'Patient', 'Control')
df_summary1 = df_summary1\
    .merge(
        mf.mean_std_simple(df_1, features=stats_meas['mean'])
    )\
    .merge(
        mf.t_test_by_group(df_1, stats_meas['mean'], group_var='Status'), on='Variable'
    )

In [8]:
# count and proportion of categorical variables

res2 = mf.count_percent(df_1, stats_meas['n'], 'Status')

df_summary2 = mf.summaryze_count_percent(df_summary, res2, stats_meas['n'], 'Patient', 'Control')
df_summary2 = df_summary2\
    .merge(
        mf.count_simple(df_1, features=stats_meas['n']), on='Variable'
    )\
    .merge(
        mf.chi_squared_tests(df_1, stats_meas['n'], group_var='Status'), on='Variable'
    )

In [9]:
# median and inter quartile range of remaining continuos variables

res3 = mf.median_iqr(df_1, stats_meas['median'], 'Status')

df_summary3 = mf.summaryze_median_iqr(df_summary, res3, stats_meas['median'], 'Patient', 'Control')
df_summary3 = df_summary3\
    .merge(
        mf.median_iqr_simple(df_1, stats_meas['median']), on='Variable'
    )\
    .merge(
        mf.mann_whitney(df_1, stats_meas['median'], 'Status'), on='Variable'
    )

In [10]:
# concatenate all results

ordered_cols = ['Variable', 'Statistical Measure', 'Patient', 'Control', 'p-value', 'Total', 'Available Sample for Analysis']
df_summary = pd.concat(
    [df_summary1, df_summary3, df_summary2], axis=0,ignore_index=True
)
df_summary = df_summary[ordered_cols].copy()

# Analysis of patients medical history

In [11]:
# rename and select second group of variables

variables_dict_2 = {
    "medical_history_neurologic___7": "Stroke",
    "medical_history_cancer___1"    : "Melanoma" ,
    "medical_history_cancer___2"    : "Prostate Cancer" ,
    "medical_history_cancer___3"    : "Other Cancers",
    "medical_history_metabolic___1" : "Diabetes",
    "medical_history_metabolic___4" : "High Cholesterol",
    "medical_history_cardiovasc___2": "Hypertension",
    "medical_history_cardiovasc___3": "History of CAD",
    "medical_history_psychologi___1": "History of Anxiety",
    "medical_history_psychologi___3": "History of Depression",
    "family_member_diagnosed_wi"    : "Family History of PD",
    "family_member_diagnosed_wi_2"  : "Family History of Tremor",
    "family_member_diagnosed"       : "Family History of Dementia"
}
variables_2 = ['participant_id', 'Status'] + list(variables_dict_2.keys())

# statistical measures
stats_meas2 = {
    'n'     : ["Stroke", "Melanoma", "Prostate Cancer", "Other Cancers", "Diabetes", "High Cholesterol", "Hypertension",
               "History of CAD", "History of Anxiety", "History of Depression", "Family History of PD", "Family History of Tremor", "Family History of Dementia"]
}

In [ ]:
# keep only variables for analysis


df_2 = df[variables_2].copy()
df_2 = df_2.rename(columns=variables_dict_2)

In [12]:
# recode variable values

df_2["Stroke"]                    = df_2["Stroke"].map({"Checked":1, "Unchecked":0})
df_2["Melanoma"]                  = df_2["Melanoma"].map({"Checked":1, "Unchecked":0})
df_2["Prostate Cancer"]           = df_2["Prostate Cancer"].map({"Checked":1, "Unchecked":0})
df_2["Other Cancers"]             = df_2["Other Cancers"].map({"Checked":1, "Unchecked":0})
df_2["Diabetes"]                  = df_2["Diabetes"].map({"Checked":1, "Unchecked":0})
df_2["High Cholesterol"]          = df_2["High Cholesterol"].map({"Checked":1, "Unchecked":0})
df_2["Hypertension"]              = df_2["Hypertension"].map({"Checked":1, "Unchecked":0})
df_2["History of CAD"]            = df_2["History of CAD"].map({"Checked":1, "Unchecked":0})
df_2["History of Anxiety"]        = df_2["History of Anxiety"].map({"Checked":1, "Unchecked":0})
df_2["History of Depression"]     = df_2["History of Depression"].map({"Checked":1, "Unchecked":0})
df_2["Family History of PD"]      = df_2["Family History of PD"].map({"Yes":1, "No":0})
df_2["Family History of Tremor"]  = df_2["Family History of Tremor"].map({"Yes":1, "No":0})
df_2["Family History of Dementia"]= df_2["Family History of Dementia"].map({"Yes":1, "No":0})

In [13]:
# create empty dataframe for summary statistics

summary_cols = ['Variable', 'Statistical Measure', 'Patient', 'Control', 'Available Sample for Analysis']
df_summary_med = pd.DataFrame(columns=summary_cols)

In [14]:
# count and proportion of categorical variables

res4 = mf.count_percent(df_2, stats_meas2['n'], 'Status')

df_summary_med = mf.summaryze_count_percent(df_summary_med, res4, stats_meas2['n'], 'Patient', 'Control')
df_summary_med = df_summary_med\
    .merge(
        mf.count_simple(df_2, features=stats_meas2['n']), on='Variable'
    )\
    .merge(
        mf.chi_squared_tests(df_2, stats_meas2['n'], group_var='Status'), on='Variable'
    )
df_summary_med = df_summary_med[ordered_cols].copy()

df_summary_med = pd.concat([
    pd.DataFrame(data=[["Medical History", "", "", "", "", "", ""]], columns=ordered_cols), df_summary_med
],
ignore_index=True, axis=0
)

In [15]:
# save result in csv file

df_4 = pd.concat([df_summary, df_summary_med], axis=0, ignore_index=True)
df_4.to_csv('/mnt/0A2AAC152AABFBB7/CGE/luxgiant-clinical/data/Table_4.csv', index=False)
df_4

,Variable,Patient,Control,p-value,Total,Available Sample for Analysis
0,Age at Recruitment (years),56.7 (12.0),50.0 (12.6),p<0.001,53.3 (12.7),6640
1,Age at Onset (years),50.0 (12.6),50.0 (12.6),1.0,50.0 (12.6),6640
2,Body Mass Index,24.2 (3.7),25.0 (3.8),p<0.001,24.6 (3.7),5979
3,Duration of Disease (years),5.0 (3.0 - 10.0),2.0 (2.0 - 11.0),0.4396,5.0 (3.0 - 10.0),3325
4,Years of Education,12.0 (8.0 - 15.0),12.0 (9.0 - 15.0),0.0149,12.0 (8.0 - 15.0),6365
5,Onset <50 years,1642 (49.5),3 (60.0),0.9812,1645 (24.8),3325
6,Onset <40 years,733 (22.1),3 (60.0),0.1331,736 (11.1),3325
7,Currently Employed,1901 (58.5),2380 (76.2),p<0.001,4281 (64.5),6371
8,Agriculture as a job,502 (15.1),383 (11.5),p<0.001,885 (13.3),6640
9,Life time direct exposure to pesticide/insecti...,613 (18.5),378 (11.4),p<0.001,991 (14.9),6639
